In [12]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

# creating video capture object for adding video file
cap = cv2.VideoCapture("test.mp4")

# Extracting video features like frame width and height and FPS
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

# Creating video writer object to write video to out_test.avi
v_recorder = cv2.VideoWriter('lane_annotated.avi', cv2.VideoWriter_fourcc('M','P','4','2'), fps, (int(frame_width),int(frame_height)))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
# Following parameters to handle the missing line cases
r_m = 0.0 #right line slope
r_c = 0.0 #right line coordinate intercept
l_m = 0.0 #left line slope
l_c = 0.0 #left line coordinate intercept
r_miss_count = 0 #count for the non detection of right line 
l_miss_count = 0 #count for the non detection of left line 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    ysize = frame.shape[0] 
    xsize = frame.shape[1]
    region_select = np.copy(frame)
    # Display the resulting frame
    cv2.imshow('Frame',frame)
    
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2YCrCb)
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 7
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    
    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = frame.shape
    vertices = np.array([[((imshape[1]/4),imshape[0]-100),((imshape[1]/2)-20, (imshape[0]/2)+80), ((imshape[1]/2)+60, (imshape[0]/2)+80), (imshape[1]-100,imshape[0]-100)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 1     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    line_image = np.copy(frame)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    left_line = [] #left line points array
    right_line = [] #Right line points array
    for line in lines:
        for x1,y1,x2,y2 in line:
            x_diff = x2 - x1
            y_diff = y2 - y1
            if (x_diff!=0):
                slope = y_diff/x_diff
                slope_in_degree = ((math.atan(slope))*180)/math.pi
                #checking for points belong to right line and adding to the right_line array     
                if (x1<(imshape[1]/2)-10):
                    if (slope_in_degree<-20)&(slope_in_degree>-45):
                        points_l = (x1,y1)
                        left_line.append(points_l)
                        points_l = (x2,y2)
                        left_line.append(points_l)
                 #checking for points belong to left line and adding to the left_line array          
                if (x1>(imshape[1]/2)-10):
                     if (slope_in_degree>20)&(slope_in_degree<45):
                        points_l = (x1,y1)
                        right_line.append(points_l)
                        points_l = (x2,y2)
                        right_line.append(points_l)
                        
    if (len(left_line)>1):#if there is detction for left line
        #applys line fitting on the detcted points
        dx,dy,x0,y0 = cv2.fitLine(np.array(left_line), cv2.DIST_L2, 0, 0.01, 0.01)
        
        slope_left = dy/dx #slope of the detected line
        c_left = y0 - (x0*slope_left) #axis intercept of the left line
        
        if(l_m!=0.0): #updating the line feature to use in the next frame 
            slope_left = (slope_left+l_m)/2
            l_m = slope_left
        else:
            l_m = slope_left
        if(l_c!=0.0):
            c_left = (c_left+l_c)/2
            l_c = c_left
        else:
            l_c = c_left
        l_miss_count = 0
        
        top_y = (int(imshape[0]/2)+100)
        bottom_y = imshape[0]
        top_x = (top_y - c_left)/slope_left
        bottom_x = (bottom_y - c_left)/slope_left
        #draw the line to frame
        cv2.line(frame,(top_x,top_y),(bottom_x,bottom_y),(0,0,255),15)
    else:#uses the last frame line parametrs if the misscount is less than 5 frame. 
        l_miss_count = l_miss_count+1
        if (l_miss_count>0 & l_miss_count<5):
            if(l_m!=0.0):
                slope_left = l_m
                c_left = l_c

                top_y = (int(imshape[0]/2)+100)
                bottom_y = imshape[0]
                top_x = (top_y - c_left)/slope_left
                bottom_x = (bottom_y - c_left)/slope_left
                cv2.line(frame,(top_x,top_y),(bottom_x,bottom_y),(0,0,255),15)
        else:
            l_c = 0.0
            l_m = 0.0
            l_miss_count = 0.0
        
    if (len(right_line)>1):
        dx,dy,x0,y0 = cv2.fitLine(np.array(right_line), cv2.DIST_L2, 0, 0.01, 0.01)
       
        slope_right = dy/dx
        c_right = y0 - (x0*slope_right)
        
        if(r_m!=0.0):
            slope_right = (slope_right+r_m)/2
            r_m = slope_right
        else:
            r_m = slope_right
        if(r_c!=0.0):
            c_right = (c_right+r_c)/2
            r_c = c_right
        else:
            r_c = c_right
        r_miss_count = 0
        
        top_y_right = (int(imshape[0]/2)+100)
        bottom_y_right = imshape[0]
        top_x_right = (top_y_right - c_right)/slope_right
        bottom_x_right = (bottom_y_right - c_right)/slope_right
        cv2.line(frame,(top_x_right,top_y_right),(bottom_x_right,bottom_y_right),(0,255,0),15)
    else:
        r_miss_count = r_miss_count+1
        if (r_miss_count>0&r_miss_count<5):
            if(r_m!=0.0):
                slope_right = r_m
                c_right = r_c

                top_y = (int(imshape[0]/2)+100)
                bottom_y = imshape[0]
                top_x = (top_y - c_right)/slope_right
                bottom_x = (bottom_y - c_right)/slope_right
                cv2.line(frame,(top_x,top_y),(bottom_x,bottom_y),(0,255,0),15)
        else:
            r_c = 0.0
            r_m = 0.0
            r_miss_count = 0.0
    
    #adding index box
    cv2.rectangle(frame,(xsize-200,30),(xsize-30,90),(255,255,255),-1)
    cv2.rectangle(frame,(xsize-190,40),(xsize-150,50),(0,0,255),-1)
    cv2.putText(frame, 'Left line', (xsize-140,50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.rectangle(frame,(xsize-190,70),(xsize-150,80),(0,255,0),-1)
    cv2.putText(frame, 'Right line', (xsize-140,80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, cv2.LINE_AA)
  
    # Draw the lines on the image
    v_recorder.write(frame) #adding frame to the video
    cv2.imshow('line',frame)
    
    #cv2.waitKey(0)
  
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release() #closes the vide capture object
v_recorder.release() #closes the video writer object
# Closes all the frames
cv2.destroyAllWindows()


